In [9]:
### On importe les différentes librairies
# import cudf
from tqdm import tqdm
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from collections import deque
import random
import time
pd.set_option('display.max_columns', None)
import random as rd
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from collections import deque
import random
import time
import datetime
pd.set_option('display.max_columns', None)
!pip install xgboost

  Using cached xgboost-1.7.3-py3-none-manylinux2014_x86_64.whl (193.6 MB)

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [10]:
# On ouvre le jeu de données
datas = pd.read_csv('./train_features_sent.csv')
labels = pd.read_csv('./train_labels_sent.csv')

In [11]:
datas["energy_consumption_per_annum"] = labels["energy_consumption_per_annum"] # Ajoute energy à la fin de datas

In [12]:
labels.head() # Output

,level_0,energy_consumption_per_annum
0,111323,290.166345
1,265182,169.818197
2,927507,144.499711
3,874620,158.742715
4,563274,82.223648


In [13]:
datas.head(1) # Input

,level_0,additional_heat_generators,additional_water_heaters,altitude,area_code,balcony_depth,bearing_wall_material,building_category,building_class,building_height_ft,building_period,building_total_area_sqft,building_type,building_use_type_code,building_use_type_description,building_year,clay_risk_level,consumption_measurement_date,has_air_conditioning,has_balcony,heat_generators,heating_energy_source,heating_type,is_crossing_building,living_area_sqft,lowe_floor_thermal_conductivity,lower_floor_adjacency_type,lower_floor_insulation_type,lower_floor_material,main_heat_generators,main_heating_type,main_water_heaters,main_water_heating_type,nb_commercial_units,nb_dwellings,nb_gas_meters_commercial,nb_gas_meters_housing,nb_gas_meters_total,nb_housing_units,nb_meters,nb_parking_spaces,nb_power_meters_commercial,nb_power_meters_housing,nb_power_meters_total,nb_units_total,outer_wall_materials,outer_wall_thermal_conductivity,outer_wall_thickness,percentage_glazed_surfaced,post_code,radon_risk_level,renewable_energy_sources,roof_material,solar_heating,solar_water_heating,thermal_inertia,upper_floor_adjacency_type,upper_floor_insulation_type,upper_floor_material,upper_floor_thermal_conductivity,ventilation_type,wall_insulation_type,water_heaters,water_heating_energy_source,water_heating_type,window_filling_type,window_frame_material,window_glazing_type,window_heat_retention_factor,window_orientation,window_thermal_conductivity,energy_consumption_per_annum
0,111323,[wood stove or insert],NaN,328.084,881,NaN,CONCRETE,[],[],13.779528,1970-1988,1291.668,House,1,Residential single-family,1979.0,low,2018-03-14,False,False,"[indeterminate oil boiler,wood stove or insert]",wood + oil,individual,crossing east west,904.1676,2.844251,FULL_EARTH,external,concrete slab,standard oil boiler,[standard oil boiler],standard oil boiler,[low temperature oil boiler],NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,solid concrete block walls,4.364981,20 et -,0.847969,45504141,low,NaN,TILES,False,False,high,EXTERNAL,INTERNAL,Wooden ceiling under metal joists,2.07826,Natural ventilation by duct,internal,[indeterminate oil boiler],oil,individual,argon or krypton,pvc,double glazing,1.116894,"[est,west]",15.347292,290.166345


In [109]:
datas.replace('', np.nan, inplace=True)
datas.replace([],np.nan,inplace=True)
datas.replace('[]',np.nan,inplace=True)
datas.replace('[NULL]',np.nan,inplace=True)
#datas.fillna("nan", inplace = True)

In [15]:
datas.head()

,level_0,additional_heat_generators,additional_water_heaters,altitude,area_code,balcony_depth,bearing_wall_material,building_category,building_class,building_height_ft,building_period,building_total_area_sqft,building_type,building_use_type_code,building_use_type_description,building_year,clay_risk_level,consumption_measurement_date,has_air_conditioning,has_balcony,heat_generators,heating_energy_source,heating_type,is_crossing_building,living_area_sqft,lowe_floor_thermal_conductivity,lower_floor_adjacency_type,lower_floor_insulation_type,lower_floor_material,main_heat_generators,main_heating_type,main_water_heaters,main_water_heating_type,nb_commercial_units,nb_dwellings,nb_gas_meters_commercial,nb_gas_meters_housing,nb_gas_meters_total,nb_housing_units,nb_meters,nb_parking_spaces,nb_power_meters_commercial,nb_power_meters_housing,nb_power_meters_total,nb_units_total,outer_wall_materials,outer_wall_thermal_conductivity,outer_wall_thickness,percentage_glazed_surfaced,post_code,radon_risk_level,renewable_energy_sources,roof_material,solar_heating,solar_water_heating,thermal_inertia,upper_floor_adjacency_type,upper_floor_insulation_type,upper_floor_material,upper_floor_thermal_conductivity,ventilation_type,wall_insulation_type,water_heaters,water_heating_energy_source,water_heating_type,window_filling_type,window_frame_material,window_glazing_type,window_heat_retention_factor,window_orientation,window_thermal_conductivity,energy_consumption_per_annum
0,111323,[wood stove or insert],NaN,328.08400,881,NaN,CONCRETE,NaN,NaN,13.779528,1970-1988,1291.6680,House,1,Residential single-family,1979.0,low,2018-03-14,False,False,"[indeterminate oil boiler,wood stove or insert]",wood + oil,individual,crossing east west,904.1676,2.844251,FULL_EARTH,external,concrete slab,standard oil boiler,[standard oil boiler],standard oil boiler,[low temperature oil boiler],NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,solid concrete block walls,4.364981,20 et -,0.847969,45504141,low,NaN,TILES,False,False,high,EXTERNAL,INTERNAL,Wooden ceiling under metal joists,2.078260,Natural ventilation by duct,internal,[indeterminate oil boiler],oil,individual,argon or krypton,pvc,double glazing,1.116894,"[est,west]",15.347292,290.166345
1,265182,NaN,NaN,360.89240,260,NaN,NaN,[condo],[2 to 11],43.635172,<1948,7136.4657,Flat,2,Residential multi-family,NaN,medium,2020-02-21,False,False,[indefinite gas boiler],gas,individual,through all way,785.7647,4.498072,LNC,uninsulated,concrete slab,standard gas boiler,"[standard gas boiler,standard gas boiler,Joule...",standard gas boiler,"[lpg/butane/propane condensing boiler,joule-ef...",0.0,24,NaN,NaN,NaN,339.0,5.0,450.0,NaN,NaN,NaN,1122.0,double solid brick walls with air gap,3.364592,20 et -,0.750366,40083759,high,NaN,NaN,False,False,very high,EXTERNAL,EXTERNAL,concrete slab,2.078260,Ventilation by opening windows,internal,[indeterminate gas boiler],gas,individual,argon or krypton,wood,double glazing,1.133072,"[est,west,sud]",17.512631,169.818197
2,927507,NaN,NaN,324.80316,573,NaN,WOOD - OTHER,NaN,NaN,19.685040,2000-2005,4197.9210,House,2,Residential multi-family,1810.0,low,2016-06-14,False,False,[standard gas boiler],gas,individual,crossing east west,1162.5012,2.813221,SANITARY_VACTILE,internal,concrete slab,standard gas boiler,[standard gas boiler],standard gas boiler,[joule-effect electric water heater],NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hollow concrete block walls,2.447516,20 et -,NaN,43247010,low,NaN,TILES,False,False,low,LNC,INTERNAL,plasterboard ceiling,1.790967,Self-regulating mechanical ventilation after 1982,internal,[standard gas boiler],gas,individual,dry air,pvc,double glazing,1.084204,"[est,west]",16.779108,144.499711
3,874620,NaN,NaN,288.71392,204,NaN,BRICKS,NaN,NaN,13.779528,<1948,1646.8767,House,1,Residential single-family,1972.0,medium,2016-10-24,False,True,[nfc electric radiant panels],electricity,individual,through all way,1937.5020,2.749757,FULL_EARTH,external,concrete slab,joule effect generators,

In [16]:
print(labels.shape)
print(datas.shape)

(1010684, 2)
(1010684, 72)


In [160]:
# Helper

def is_categorical(array_like): # Text or mixed numeric and non-numeric values
    return array_like.dtype.name == 'object'

def is_numerical(array_like):
    return (array_like.dtype.name == 'int64' or array_like.dtype.name == 'float64')

def is_bool(array_like):
    return(array_like.dtype.name == 'bool')

In [204]:
i=-1
def change_cell(cell):
    global i
    i+=1
    return (cell,i)

def cleandata(a, b):
    datas = pd.read_csv('./train_features_sent.csv')
    labels = pd.read_csv('./train_labels_sent.csv')
    datas.replace('', np.nan, inplace=True)
    datas.replace([],np.nan,inplace=True)
    datas.replace('[]',np.nan,inplace=True)
    datas.replace('[NULL]',np.nan,inplace=True)
    clean_data = datas.drop(["building_period", "building_use_type_code", "is_crossing_building", "area_code", "post_code", "level_0"], axis=1)
    clean_data = clean_data.iloc[a:b]
    
  # N = 25 # seuil d'instabilite
  # for index, row in clean_data.iterrows():
  #     if row.isna().sum() >= N:
  #       #print(index)
  #       clean_data.drop(index, axis = 0, inplace = True)
  # Déjà fait
    
    categorical_columns = []
    for index,col in enumerate(clean_data.columns): 
        if is_bool(clean_data[col]):
            pass
        elif is_numerical(clean_data[col]):
            clean_data[col].fillna(clean_data[col][0:1000].median(), inplace = True) 
            pass
          # Remplace les NaN par la médiane des 1000 premiers éléments
            clean_data[col] = (clean_data[col] -clean_data[col][0:1000].mean())/ clean_data[col][0:1000].std()
        # Normalise
        else:
            categorical_columns.append(str(datas.columns[index]))
            clean_data[col].apply(change_cell)

    clean_data = clean_data.append(datas.iloc[0], ignore_index=True)
    clean_data = pd.get_dummies(clean_data, columns=categorical_columns, drop_first=True)
  # Convert categorical variable into dummy/indicator variables. Generates binary vectors for each categorical col
  # in df. Removes redundance ie: each out col will correspond to unique category. Numerical and boolean unchanged.
  # 1. clean_data: Data of which to get dummy indicators. 
  # 2. columns=columns_categorical: Column names in the DataFrame to be encoded. 
  # in k diff col -> out k lines (0...k-1)
  # 3. drop_first=True: Get k-1 dummies out of k categorical levels by removing the first level.
  # in k diff col -> k lines (0,...k-2), 
    datas["energy_consumption_per_annum"] = labels["energy_consumption_per_annum"] # Ajoute energy à la fin de datas
    return clean_data

In [205]:
x_train = cleandata(0, 10)
x_test  = cleandata(10, 20)
a = np.zeros(len(x_train))
for col in x_train.columns :
    try :
        x_test.col 
    except Exception as e : 
        x_test[col] = a
for col in x_test.columns :
    try :
        x_train.col 
    except Exception as e : 
        x_train[col] = a
        
        

print(x_train.shape, x_test.shape)

(11, 217) (11, 217)


In [206]:
x_train.head()

,additional_water_heaters,building_category,building_class,building_total_area_sqft,building_use_type_description,has_balcony,heat_generators,lower_floor_material,main_heat_generators,main_heating_type,main_water_heaters,main_water_heating_type,nb_commercial_units,nb_dwellings,nb_gas_meters_commercial,nb_gas_meters_housing,nb_gas_meters_total,nb_housing_units,nb_meters,nb_power_meters_commercial,nb_power_meters_total,outer_wall_thickness,percentage_glazed_surfaced,solar_heating,water_heaters,water_heating_type,window_filling_type,window_frame_material,window_glazing_type,window_heat_retention_factor,window_orientation,window_thermal_conductivity,altitude_-0.7986212086837555,altitude_-0.7787550094627665,altitude_-0.49069512075842686,altitude_-0.25230073010655957,altitude_-0.1430366343911203,altitude_-0.13310353478062578,altitude_-0.03377253867568102,altitude_1.4462593032879958,altitude_2.1117769771911257,altitude_328.084,bearing_wall_material_CONCRETE,bearing_wall_material_GRITSTONE,bearing_wall_material_INDETERMINED,bearing_wall_material_STONE,bearing_wall_material_WOOD - OTHER,building_height_ft_-0.7737041736469743,building_height_ft_-0.7058353864849589,building_height_ft_-0.3664914506748826,building_height_ft_-0.16288508918883682,building_height_ft_-0.09501630202682146,building_height_ft_0.48186838885030814,building_height_ft_0.7872779310793769,building_height_ft_2.3821944293867356,building_height_ft_13.779528,building_type_House,building_year_-0.08513387896883273,building_year_0.03405355158753309,building_year_0.15324098214389892,building_year_0.45972294643169676,building_year_0.5789103769880626,building_year_0.6810710317506619,building_year_0.715124583338195,building_year_1979.0,clay_risk_level_medium,consumption_measurement_date_2014-03-17,consumption_measurement_date_2015-09-07,consumption_measurement_date_2016-06-14,consumption_measurement_date_2016-10-24,consumption_measurement_date_2018-03-14,consumption_measurement_date_2018-04-29,consumption_measurement_date_2019-01-21,consumption_measurement_date_2019-07-23,consumption_measurement_date_2020-02-21,heating_energy_source_fuel oil,heating_energy_source_gas,heating_energy_source_wood + electricity,heating_energy_source_wood + oil,living_area_sqft_-0.8711133950750527,living_area_sqft_-0.5726010167004241,living_area_sqft_-0.43691357198468367,living_area_sqft_-0.11126370466690721,living_area_sqft_0.07869871793512888,living_area_sqft_0.35007360736660964,living_area_sqft_1.2998857203767913,living_area_sqft_2.0325979218417882,living_area_sqft_904.1676,lowe_floor_thermal_conductivity_-0.6956163027963149,lowe_floor_thermal_conductivity_-0.6183672356710878,lowe_floor_thermal_conductivity_-0.5619629514715087,lowe_floor_thermal_conductivity_-0.5434418045023649,lowe_floor_thermal_conductivity_-0.5068075738032839,lowe_floor_thermal_conductivity_1.4457069768250081,lowe_floor_thermal_conductivity_2.844250661062471,lower_floor_adjacency_type_LNC,lower_floor_adjacency_type_SANITARY_VACTILE,lower_floor_insulation_type_internal,lower_floor_insulation_type_uninsulated,outer_wall_materials_hollow brick walls,outer_wall_materials_hollow concrete block walls,outer_wall_materials_solid concrete block walls,outer_wall_thermal_conductivity_-0.47961110722683015,outer_wall_thermal_conductivity_-0.3738987006015383,outer_wall_thermal_conductivity_-0.33153439250782624,outer_wall_thermal_conductivity_-0.2890898435860251,outer_wall_thermal_conductivity_-0.010155777846231735,outer_wall_thermal_conductivity_0.5019475937943962,outer_wall_thermal_conductivity_2.6481357547262636,outer_wall_thermal_conductivity_4.364981193209637,radon_risk_level_low,radon_risk_level_medium,roof_material_OTHERS,roof_material_TILES,solar_water_heating_True,thermal_inertia_low,thermal_inertia_medium,thermal_inertia_very high,upper_floor_adjacency_type_LNC,upper_floor_insulation_type_INTERNAL,upper_floor_material_Wooden ceiling under wooden joists,upper_floor_material_attic space under slope,upper_floor_material_ceiling between 

In [207]:
x_test.head()

,additional_water_heaters,building_category,building_class,building_total_area_sqft,building_use_type_description,has_balcony,heat_generators,lower_floor_material,main_heat_generators,main_heating_type,main_water_heaters,main_water_heating_type,nb_commercial_units,nb_dwellings,nb_gas_meters_commercial,nb_gas_meters_housing,nb_gas_meters_total,nb_housing_units,nb_meters,nb_power_meters_commercial,nb_power_meters_total,outer_wall_thickness,percentage_glazed_surfaced,solar_heating,water_heaters,water_heating_type,window_filling_type,window_frame_material,window_glazing_type,window_heat_retention_factor,window_orientation,window_thermal_conductivity,additional_heat_generators_[wood stove or insert],altitude_-0.8749612539433479,altitude_-0.8087479698611487,altitude_-0.5438948335323514,altitude_-0.48714059003332344,altitude_0.21282841312135486,altitude_0.34525498128575366,altitude_1.3195361613524008,altitude_1.877619555759509,altitude_328.084,bearing_wall_material_AGGLOMERATE - CONCRETE,bearing_wall_material_BRICKS,bearing_wall_material_CONCRETE,bearing_wall_material_WOOD - BRICK,building_height_ft_-0.8130611252316831,building_height_ft_-0.7637846933994598,building_height_ft_0.024638215916111914,building_height_ft_0.2217439432450051,building_height_ft_0.36957323874167447,building_height_ft_0.5666789660705671,building_height_ft_0.8130611252316836,building_height_ft_1.2072725798894695,building_height_ft_13.779528,building_type_House,building_year_-0.8981144055009389,building_year_-0.7135703495760872,building_year_-0.15993818180153246,building_year_0.2706646153564546,building_year_0.5167233565895901,building_year_0.8242967831310094,building_year_1.008840839055861,building_year_1979.0,clay_risk_level_low,clay_risk_level_medium,consumption_measurement_date_2014-09-11,consumption_measurement_date_2014-11-04,consumption_measurement_date_2015-03-30,consumption_measurement_date_2016-05-25,consumption_measurement_date_2018-03-14,consumption_measurement_date_2018-04-26,consumption_measurement_date_2018-08-24,consumption_measurement_date_2018-11-08,consumption_measurement_date_2019-02-15,consumption_measurement_date_2020-02-27,has_air_conditioning_True,heating_energy_source_gas,heating_energy_source_wood + electricity,heating_energy_source_wood + electricity + fuel oil,heating_energy_source_wood + oil,living_area_sqft_-0.9938830646706558,living_area_sqft_-0.7352517839013399,living_area_sqft_-0.3288311998352722,living_area_sqft_-0.2549365481868968,living_area_sqft_0.18843136170335864,living_area_sqft_0.22537868752754714,living_area_sqft_0.44706264247267485,living_area_sqft_1.2599038106048097,living_area_sqft_1.7402190463192535,living_area_sqft_904.1676,lowe_floor_thermal_conductivity_-0.7167212861697287,lowe_floor_thermal_conductivity_-0.6344149470819704,lowe_floor_thermal_conductivity_-0.5013691680107342,lowe_floor_thermal_conductivity_0.09757500979741686,lowe_floor_thermal_conductivity_0.5127949238215077,lowe_floor_thermal_conductivity_0.9192996424978184,lowe_floor_thermal_conductivity_2.3136057995082684,lowe_floor_thermal_conductivity_2.844250661062471,lower_floor_adjacency_type_FULL_EARTH,lower_floor_adjacency_type_LNC,lower_floor_adjacency_type_OUTSIDE,lower_floor_adjacency_type_SANITARY_VACTILE,lower_floor_insulation_type_internal,lower_floor_insulation_type_uninsulated,outer_wall_materials_hollow brick walls,outer_wall_materials_hollow concrete block walls,outer_wall_materials_solid concrete block walls,outer_wall_thermal_conductivity_-0.7803226715818881,outer_wall_thermal_conductivity_-0.7232388363847388,outer_wall_thermal_conductivity_-0.4377432628235198,outer_wall_thermal_conductivity_-0.3233307466028853,outer_wall_thermal_conductivity_-0.26604657629913114,outer_wall_thermal_conductivity_-0.15128948475567017,outer_wall_thermal_conductivity_1.7430503898733907,outer_wall_thermal_conductivity_1.927638891953862,outer_wall_thermal_conductivity_4.364981193209637,radon_risk_level_low,radon_risk_level_medium,roof_material_SLATE,roof_material_T

In [208]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
clean.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in clean.columns.values]
clean.columns.values # Converts to Object array and removes [ ] when necessary

array(['altitude', 'building_height_ft', 'building_total_area_sqft', ...,
       'window_orientation__undetermined,west_',
       'window_orientation__west,sud_', 'window_orientation__west_'],
      dtype=object)

In [ ]:
clean.head()

In [21]:
y_train = clean["energy_consumption_per_annum"]
X_train = clean.drop(["energy_consumption_per_annum"],axis=1)
# axis = 0 or index, 1 or columns -> Whether to drop labels from index or columns.

In [35]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'subsample': [0.45, 0.5, 0.55], 'n_estimators': [1200, 1400], 
     'max_depth': [5], 'learning_rate': [0.02],
     'colsample_bytree': [0.4], 'colsample_bylevel': [0.5],
     'reg_alpha':[1], 'reg_lambda': [1], 'min_child_weight':[2]}
]

# n estimators : plus il est élevé plus il y a d'overfit. 100 pas bcp, 200 ou 300 bien, 1200 -> 1
# 

xgb_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)



In [ ]:
xgb_model.fit(X_train, y_train)

In [ ]:
score = xgb_model.score(X_train, y_train)
print(score)

In [ ]:
grid_search = GridSearchCV(
    xgb_model,
    param_grid=param_grid,
    scoring = 'neg_root_mean_squared_error',
    n_jobs = -1,
    cv = 5,
    verbose=True
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_score_)


In [ ]:
grid_search.best_estimator_.score(X_train,y_train)

# code ancien

In [ ]:
# Remove row 10 building period redondance avec building year
# Remove row 12 building type code (res/com) car redondance (row 12 après delete)
# Cast row 13 to int (1973.5)

datas.replace('', str(np.nan), inplace=True)
datas.replace([], str(np.nan), inplace=True)
datas.replace('[]', str(np.nan), inplace=True)
datas.replace('[NULL]', str(np.nan), inplace=True)
datas.replace(np.nan, str(np.nan), inplace=True)
datas.drop(["building_period", "building_use_type_code", "is_crossing_building"], axis=1, inplace=True)

datas.head(50)

In [ ]:
"""
def is_nan(val_array):
    c = val_array.eq("nan").sum()
    return c

def is_date(date_str):
    format = "%Y-%m-%d"
    return bool(datetime.datetime.strptime(date_str, format))

def is_negative(col_nums, row):
    c=0
    for num in col_nums:
        if row[num] != 'nan' and row[num] <= 0:
            c+=1
    return c

def is_snegative(col_nums, row):
    c=0
    for num in col_nums:
        if row[num] != 'nan' and row[num] < 0:
            c+=1
    return c

def is_not_bool(col_nums, row):
    c=0
    for num in col_nums:
        if (row[num] != 'nan') and (not isinstance(row[num], bool)): 
            c += 1
    return c

def not_in_lmhvh(col_nums, row):
    c=0
    for num in col_nums:
        if row[num] != 'nan' and (row[num] not in ["low", "medium", "high", "very high"]):
            c += 1
    return c

def enough_data(row):
    counter = 0

    counter += is_nan(row)

    if (row["area_code"] != 'nan' and (row["area_code"] < 0 or row["area_code"] > 999)): 
        counter += 1
    
    if row["bearing_wall_material"] != 'nan' and (row["bearing_wall_material"] == "INDETERMINED"): 
        counter += 1
    
    
    counter += is_negative(["altitude", "building_height_ft", "building_total_area_sqft", "living_area_sqft", "lowe_floor_thermal_conductivity", "window_heat_retention_factor", "window_thermal_conductivity"], row)
    
    counter += is_snegative(["nb_commercial_units", "nb_dwellings", "nb_gas_meters_commercial", "nb_gas_meters_housing", "nb_gas_meters_total", "nb_housing_units", "nb_meters", "nb_parking_spaces", "nb_power_meters_commercial", "nb_power_meters_housing", "nb_power_meters_total", "nb_units_total", "outer_wall_thermal_conductivity", "upper_floor_thermal_conductivity", "percentage_glazed_surfaced"], row) 

    if row["building_year"] != 'nan' and (row["building_year"] <= 1300 or row["building_year"] >= 2023):
        counter += 1
    
    counter += not_in_lmhvh(["clay_risk_level", "radon_risk_level", "thermal_inertia"], row)
    
    if row["consumption_measurement_date"] != 'nan' and not is_date(row["consumption_measurement_date"]): 
        counter += 1
    
    counter += is_not_bool(["has_air_conditioning", "has_balcony", "solar_heating", "solar_water_heating"], row) 
    
    if row["heating_type"] != 'nan' and row["heating_type"] not in ['individual' 'collective']: 
        counter += 1
    
    if row["lower_floor_adjacency_type"] != 'nan' and row["lower_floor_adjacency_type"] not in ['FULL_EARTH', 'LNC', 'SANITARY_VACTILE', 'OUTSIDE', 'ADJACENT BUILDING', 'BURIED_WALL']:
        counter += 1
    
    if row["upper_floor_adjacency_type"] != 'nan' and row["upper_floor_adjacency_type"] not in ['EXTERNAL', 'LNC', 'ADJACENT_BUILDING', 'INNER_WALL']: 
        counter += 1
    
    if row["water_heating_type"] != 'nan' and row["water_heating_type"] not in ['individual', 'collective']: 
        counter += 1
    if counter >=10 : 
        datas.drop([i], inplace = True)
"""

In [ ]:
counter=0
from tqdm import tqdm
tqdm(datas.apply(enough_data, axis = 0))

In [ ]:
#Nous remplaçons toutes les valeures nulles par la même chaîne de caractères afin de pouvoir les manipuler 
datas.replace('', 'nan', inplace = True)
datas.replace([], 'nan', inplace = True)
datas.replace('[]', 'nan', inplace = True)
datas.replace('NULL', 'nan', inplace = True)

In [ ]:
### Définissons une fonction qui permet de traiter les valeurs en str en comptant les occurences de chaque terme

L = []
def format_col(col, Values):
    L = []
    for c in datas[col]:
        a = np.zeros(len(Values))
        for i in range(len(Values)) : 
            print(c)
            print(c.count(Values[i]))
            b = c.count(Values[i])
            a[i] = b
        L.append(a)
    datas[col] = L


In [ ]:
To_formate_manually = ['building_category', 'building_class', 'main_heating_type', 'main_water_heating_type' ]
for column in datas.columns[1:] : 
    if (datas['{}'.format(column)].dtype != str):
        datas['{}'.format(column)] = datas['{}'.format(column)].astype(str)
        enc = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')
        X = datas['{}'.format(column)] 
        X = np.array(X).reshape(-1,1)
        enc.fit(X)
        datas['{}'.format(column)] = list(enc.transform(X))
format_col('building_category', ['condo', 'individual house'])
format_col('building_class', ['2 to 11', 'individual', '12+'])
format_col('main_heating_type', ['standard gas boiler', 'low temperature gas boiler', 'gas condensing boiler', 'electric boiler', 'Joule effect generators', 'solar heating'])
format_col('main_water_heating_type', ['low temperature oil boiler','lpg/butane/propane condensing boiler','joule-effect electric water heater','solar hot water','low temperature gas boiler','gas condensing boiler'])

###datas.head()

In [ ]:
datas.to_csv('out.csv', index=False)

In [ ]:
32.
### Here are some tests 

def isnan(array_value): 
    c=0
    for val in array_value :
        if (val.isnan()) :
            C+=1
    return c
def is_date(date_str):
    format = "%d-%m-%Y"
    return bool(datetime.strptime(date_str, format))

def enough_data(row):
    counter=0
    counter+=is_nan(row)
 
    if row[3] != nan && row[3] <= 0:  # altitude, pbp
        counter += 1
    if row[4] != nan && (row[4] < 0 || row[4]) > 999: # area code, pbp
        counter += 1
    if row[6] != nan && (row[6] == "INDETERMINED" || row[6]) == "OTHER": # bearing_wall_material
        counter += 1
    if row[9] != nan && row[9] <= 0: # building height, pbp
        counter += 1
    if row[10] != nan && row[10] <= 0: # building total area sqft, pbp
        counter += 1
    if row[13] != nan && (row[13] <= 1300 || row[13] >= 2023): # building year
        counter += 1
    if row[14] != nan && (row[14] not in ["low",medium",high"]):
        counter += 1
    if row[15] != nan && not(is_date(row[15])): # consumption_measurement_date
        counter += 1
    if row[16] != nan && not isinstance(row[16], bool): # has_air_conditioning
        counter += 1
    if row[17] != nan && not isinstance(row[16], bool): # has_balcony
        counter += 1
    if row[20] != nan && row[20] not in ['individual' 'collective']: # heating type
        counter += 1
    
    
    


In [ ]:
### Standardising datas

for column in datas.columns : 
    if (column != 'level_0'):
        datas['{}'.format(column)] -= datas['{}'.format(column)].mean()
        datas['{}'.format(column)] /= datas['{}'.format(column)].std()
labels['energy_consumption_per_annum'] -= labels['energy_consumption_per_annum'].mean()
labels['energy_consumption_per_annum'] /= labels['energy_consumption_per_annum'].std()

In [ ]:
### Merging datas to shuffle
datas['energy_consumption_per_annum'] = labels['energy_consumption_per_annum']

In [ ]:
### On drop les ID qui n'apportent pas d'information sur la consommation

datas = datas.drop('level_0', axis = 1)

### On shuffle pour bien mélanger les données

shuffle(datas)

### On récupère les labels dans le nouvel ordre 

labels['energy_consumption_per_annum'] = datas['energy_consumption_per_annum']

### On enlève les labels des données d'entraînement et on récupère X et y dans des vecteurs 

datas = datas.drop('energy_consumption_per_annum', axis = 1)
y_train = labels['energy_consumption_per_annum']
X_train = np.array(datas.transpose()).reshape(1010684,70,1)


In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint 
import sklearn
from sklearn.metrics import explained_variance_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
model = Sequential()

model.add(LSTM(128,input_shape=(X_train.shape[1:]), return_sequences = True, activation = 'tanh'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences = True, activation = 'tanh'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, activation = 'tanh'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation = 'softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.001, decay = 1e-6)
model.compile(loss = 'mse', optimizer = opt, metrics = ['accuracy'])

In [ ]:
call_list = [tf.keras.callbacks.EarlyStopping(patience = 100)]

History = model.fit(X_train, y_train, 
                    validation_data = (X_test, y_test)
                   batch_size = 32,
                   epochs = 10,
                   callbacks = call_list )

In [ ]:
y_pred = model.predict(X_test)
Explained_variance = sklearn.metrics.explained_variance_score
print(Explained_variance)

In [ ]:
### Ici on peut tenter une PCA
from sklearn.decomposition import PCA 
pca = PCA(n_components = 10)
pca.fit(datas)
x_pca = pca.transform(datas)

In [ ]:
y_train = labels['energy_consumption_per_annum']
X_train = np.array(datas.transpose()).reshape(1010684,70)

In [ ]:


# First, split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

# Next, create an instance of the RandomForestRegressor class
# you can specify the number of trees in the forest with the n_estimators parameter
# you can specify the number of features to consider at each split with max_features parameter
# you can set the random_state to get reproducible results
rf = RandomForestRegressor(n_estimators=100, max_features='sqrt',random_state=0)

# Fit the model to the training data
rf.fit(X_train, y_train)

# Use the model to predict on the testing data
y_pred = rf.predict(X_test)

# Evaluate the model's performance using metrics such as mean absolute error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from tqdm import tqdm
# First, split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

# Next, create an instance of the RandomForestRegressor class
rf = RandomForestRegressor(n_estimators=100, max_features='sqrt',random_state=0)

rf.fit(X_train, y_train)
 
# Use the model to predict on the testing data
y_pred = rf.predict(X_test)

# Evaluate the model's performance using metrics such as mean absolute error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)

In [ ]:
'[condo,condo,condo,condo,condo,condo,condo,condo,condo,condo]'.count('condo')